In [1]:
# Only needed in pycharm
import os

os.chdir('/home/cs-folq1/src/sts_bert/sBERT')

In [2]:
import torch

from Datasets import load_wiki_qa
from WikiQA.WikiQAClassifierTrainer import WikiQAClassifierTrainer
from WikiQA.WikiQABinaryClassifierTrainer import WikiQABinaryClassifierTrainer
from CrossEncoder import CrossEncoder

wiki_qa = load_wiki_qa()

Using custom data configuration default
Reusing dataset wiki_qa (/home/cs-folq1/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c)


In [ ]:
original_sts_cross_model_path = \
    'results/pretraining_sts/results_210702_094435_cross-encoder_cls-pooling-hidden_best_model.bin'
original_nli_sts_cross_model_path = \
    'results/pretraining_sts/results_210702_094435_cross-encoder_nli-base_best_model.bin'
original_sts_bi_model_path = \
    'results/pretraining_sts/results_210707_152607_bi-encoder_base-cls-pooling_best_model.bin'
original_nli_sts_bi_model_path = \
    'results/pretraining_sts/results_210707_152607_bi-encoder_nli-cls-pooling_best_model.bin'

In [ ]:
# Train on PAWS.

def run_experiment(config):
    encoder, pretrained_model = config['pretrained_model'].split('/')

    if encoder == 'cross':
        if pretrained_model in ['bert', 'sts']:
            sts_model = CrossEncoder(mode='cls-pooling-hidden')
            if pretrained_model == 'sts':
                print('Loading sts pretrained model from ' + original_sts_cross_model_path)
                sts_model.load_state_dict(torch.load(original_sts_cross_model_path))
        else:
            assert pretrained_model in ['nli', 'nli-sts']:
            sts_model = CrossEncoder(mode='nli-base')
            if pretrained_model == 'nli-sts':
                print('Loading nli and sts pretrained model from ' +
                    original_nli_sts_cross_model_path)
                sts_model.load_state_dict(torch.load(original_nli_sts_cross_model_path))
        
        if pretrained_model in ['bert', 'nli']:
            paws_model = CrossEncoderPretrained(sts_model, mode='as-is')
        else:
            assert(pretrained_model in ['sts', 'nli-sts'])
            paws_model = CrossEncoderPretrained(sts_model, mode='replace-head')
    else:
        assert(encoder == 'bi')
        if pretrained_model == 'nli':
            paws_model = BiEncoder(mode='nli-linear-pooling', head='extra-head-sub')
        else:
            paws_model = BiEncoder(mode='base-linear-pooling', head='extra-head-sub')
            if pretrained_model == 'nli-sts':
                print('Loading nli and sts pretrained model from ' + original_nli_sts_bi_model_path)
                paws_model.load_state_dict(torch.load(original_nli_sts_bi_model_path))
            elif pretrained_model == 'sts':
                print('Loading sts pretrained model from ' + original_sts_bi_model_path)
                paws_model.load_state_dict(torch.load(original_sts_bi_model_path))
            else:
                assert(pretrained_model == 'bert')


    trainer = PawsTrainer(model=paws_model, train_dataset=train_dataset_subset,
                          dataset={'dev': dev_dataset_subset, 'test': wiki_qa['test']},
                          num_epochs=config['num_epochs'], batch_size=config['batch_size'],
                          lr=config['lr'])
    result = trainer.train(disable_progress_bar=False, eval_zero_shot=True)
    save_name = 'pretrained_' + config['pretrained_model']
    return result, paws_model, save_name


grid = {
    'num_epochs': 10,  # Size 5000 => 50s per epoch????
    'batch_size': 16,
    'lr': 2e-5,
    'pretrained_model': [
        'cross/bert', 'cross/nli', 'cross/sts', 'cross/nli-sts',
        'bi/bert', 'bi/nli', 'bi/sts', 'bi/nli-sts'],
    # 'mode': ['replace-head', 'shift-bias', 'additional-head'],
    # 'mode': 'shift-bias',
    # 'train_size': 2000,
    'train_size': [10, 50, 100, 500, 1000, 2000, 5000, 10000],
    'train_subset_seed': [1, 2, 3]
}

#grid_run = GridRun(run_experiment, results_dir='results', experiment_name='paws_from_nli_sts')
grid_run = GridRun(run_experiment)
grid_run.run(grid, save_best=False)

In [5]:
model = CrossEncoder(mode='mean-pooling')
trainer = WikiQABinaryClassifierTrainer(model=model, dataset=wiki_qa, mode='all-unscaled',
                                        trainset_size=20000, trainset_seed=1, num_epochs=3,
                                        batch_size=16, lr=2e-5, devset_size=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scheduler type: constant_with_warmup, epochs: 3, steps per epoch: 1250, total steps: 3750, warmup steps: 0


In [6]:
trainer.train(disable_progress_bar=False, eval_zero_shot=True, early_stopping=False)

  0%|          | 1/1250 [00:00<02:56,  7.09it/s]

       Predicted-  Predicted+
Gold-         690        1903
Gold+          28         112
Precision 0.06, recall 0.80, F1 0.10
Epoch 0   : loss: 0.7227  , score: 0.1039  *


  0%|          | 1/1250 [00:00<03:21,  6.21it/s]

       Predicted-  Predicted+
Gold-        2583          10
Gold+         127          13
Precision 0.57, recall 0.09, F1 0.16
Epoch 1   : loss: 0.1475  , score: 0.1595  *


  0%|          | 1/1250 [00:00<02:24,  8.63it/s]

...

  1%|          | 3/386 [00:00<00:18, 20.65it/s]

       Predicted-  Predicted+
Gold-        2541          52
Gold+          88          52
Precision 0.50, recall 0.37, F1 0.43
Epoch 3   : loss: 0.1704  , score: 0.4262  


100%|██████████| 386/386 [00:19<00:00, 19.30it/s]

       Predicted-  Predicted+
Gold-        5770         102
Gold+         198          95
Precision 0.48, recall 0.32, F1 0.39
Test loss: 0.1588, score: 0.3878


(0.38775510204081637, tensor(0.1588))

In [15]:
trainer.predict_batch({'question': [wiki_qa['validation'][i]['question'] for i in range(10)],
                       'answer': [wiki_qa['validation'][i]['answer'] for i in range(10)],
                       'label': [wiki_qa['validation'][i]['label'] for i in range(10)],
                       'question_number': [0] * 10
                       })

(tensor([2.5538e-03, 5.6121e-04, 5.7433e-02, 7.5890e-01, 1.0651e-02, 1.2402e-03,
         1.6844e-03, 5.5506e-04, 4.8925e-04, 1.6593e-01], device='cuda:0',
        grad_fn=<DivBackward0>),
 [(0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0),
  (0.0, 0)])

In [17]:
[wiki_qa['validation'][i] for i in range(10, 20)]

[{'answer': 'Headquartered in Houston , Texas , BMC develops, markets and sells software used for multiple functions, including IT service management, data center automation, performance management, virtualization lifecycle management and cloud computing management.',
  'document_title': 'BMC Software',
  'label': 0,
  'question': 'how big is bmc software in houston, tx',
  'question_id': 'Q11'},
 {'answer': 'The name "BMC" is taken from the surnames of its three founders—Scott Boulette, John Moores, and Dan Cloer.',
  'document_title': 'BMC Software',
  'label': 0,
  'question': 'how big is bmc software in houston, tx',
  'question_id': 'Q11'},
 {'answer': 'Employing over 6,000, BMC is often credited with pioneering the BSM concept as a way to help better align IT operations with business needs.',
  'document_title': 'BMC Software',
  'label': 1,
  'question': 'how big is bmc software in houston, tx',
  'question_id': 'Q11'},
 {'answer': 'For 2011, the company recorded an annual reven

In [6]:
del model
del trainer
torch.cuda.empty_cache()

In [11]:
import numpy as np

#dev_pred = trainer.predict(trainer.dev_dl)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(np.array(dev_pred[1])[:,0], np.array(dev_pred[0]) > 0.5)
#dev_pred


In [15]:
import pandas as pd
print(pd.DataFrame(cm, columns=['Predicted-', 'Predicted+'], index=['Gold-', 'Gold+']))

       Predicted-  Predicted+
Gold-        1760         833
Gold+          84          56


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'best_dev_performance',
 'best_model',
 'best_model_epoch',
 'debug_step_function',
 'dev_dl',
 'loss_function',
 'lr_scheduler',
 'model',
 'num_epochs',
 'optimizer',
 'performance',
 'predict',
 'predict_batch',
 'score',
 'sigmoid',
 'test_dl',
 'train',
 'train_dl']